# **My First ML Project**


## **Cell 1: Setup and Data Generators**

In [ ]:
# Import necessary libraries
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.optimizers import Adam
import os

# Check GPU availability
print("Num GPUs Available:", len(tf.config.list_physical_devices('GPU')))

# Define image dimensions, batch size, and number of epochs
IMG_HEIGHT = 224
IMG_WIDTH = 224
BATCH_SIZE = 32
EPOCHS = 5

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Set dataset directories (adjust these paths if needed)
dataset_path = '/content/drive/MyDrive/butterfly_vs_moth'
train_dir = '/content/drive/MyDrive/butterfly_vs_moth/train'
validation_dir = '/content/drive/MyDrive/butterfly_vs_moth/validation'
model_path = '/content/drive/MyDrive/butterfly_vs_moth_model.h5'

#  Reload Model If It Exists (Prevents Retraining from Scratch)
if os.path.exists(model_path):
    print(" Loading saved model...")
    model = load_model(model_path)
    print(" Model loaded successfully!")
else:
    print(" No saved model found. Training a new model...")


# Create ImageDataGenerators for training and validation
train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    rotation_range=20,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

validation_datagen = ImageDataGenerator(rescale=1.0/255)

# Flow images from directories
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

# Print number of classes
num_classes = train_generator.num_classes
print("Number of classes:", num_classes)



## **Cell 2: Build and Train the Model**

In [ ]:
# Build the model using transfer learning with MobileNetV2
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(IMG_HEIGHT, IMG_WIDTH, 3))
base_model.trainable = False  # Freeze the base model for faster training

# Add custom layers on top of the base model
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
# Final layer with softmax activation for 10 classes
predictions = Dense(num_classes, activation='softmax')(x)

# Create the complete model
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // BATCH_SIZE,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // BATCH_SIZE,
    epochs=EPOCHS
)

# Save the trained model to Google Drive so it persists beyond your session
model_save_path = '/content/drive/MyDrive/butterfly_vs_moth_model.h5'
model.save(model_save_path)
print("Model training complete and saved to", model_save_path)


### **Cell 3: Inference Example**

In [ ]:
# Import additional libraries for inference
from tensorflow.keras.preprocessing import image
import numpy as np

# Load a sample image from the validation set (change index as needed)
sample_img_path = validation_generator.filepaths[0]   # First image in the validation set
print("Sample image path:", sample_img_path)

# Load and preprocess the image
img = image.load_img(sample_img_path, target_size=(IMG_HEIGHT, IMG_WIDTH))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0) / 255.0

# Predict the class of the sample image
preds = model.predict(img_array)
predicted_class = np.argmax(preds, axis=1)
print("Predicted class index:", predicted_class)

# Display the mapping from class indices to folder names
print("Class mapping:", train_generator.class_indices)
